In [1]:
import matplotlib.pyplot as plt
import numpy as np
from keras.callbacks import EarlyStopping, TensorBoard
from keras.layers import Input, Dense
from keras.models import Model
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.backend import mean, square

from spektral.datasets import qm9
from spektral.layers import EdgeConditionedConv, GlobalAttentionPool
from spektral.utils import label_to_one_hot

import psi4

import time
from os import path, getcwd

Using TensorFlow backend.


In [2]:
'''
The following is setup code for the neural calculation
as well as allowing for actual lookups to compare accuracies.
'''

A_complete, X_complete, E_complete, y_complete = qm9.load_data(return_type='numpy',
                           nf_keys='atomic_num',
                           ef_keys='type',
                           self_loops=True,
                           amount=None)  # Set to None to train on whole dataset
# one-hot labeling of atoms
uniq_X = np.unique(X_complete)
X_complete = label_to_one_hot(X_complete, uniq_X)

clusters = [['A', 'B', 'alpha'], 
               ['C', 'r2', 'u0'],
               ['zpve', 'g298', 'cv'],
               ['lumo', 'u298', 'h298'],
               ['mu', 'homo']]

N = X_complete.shape[-2]           # Number of nodes in the graphs
F = X_complete.shape[-1]           # Node features dimensionality
S = E_complete.shape[-1]           # Edge features dimensionality

X_in = Input(shape=(N, F))
A_in = Input(shape=(N, N))
E_in = Input(shape=(N, N, S))

Loading QM9 dataset.
Reading SDF


100%|██████████| 133885/133885 [00:40<00:00, 3328.25it/s]


In [3]:
def create_hard_parameter_sharing_model(num_tasks=1):
    gc1 = EdgeConditionedConv(64, activation='relu')([X_in, A_in, E_in])
    gc2 = EdgeConditionedConv(128, activation='relu')([gc1, A_in, E_in])
    pool = GlobalAttentionPool(256)(gc2)
    dense_list = [Dense(256, activation='relu')(pool) for i in range(num_tasks)]
    output_list = [Dense(1)(dense_layer) for dense_layer in dense_list]
    return Model(inputs=[X_in, A_in, E_in], outputs=output_list)

In [4]:
def generate_model_filename(tasks):
    tasks_str = "".join(sorted(tasks))
    return path.join('demo_models', tasks_str + '.h5')

In [5]:
def generate_model_helper_filename(task):
    return path.join('demo_models', task + '.txt')

In [6]:
def predict_property_neural(prop=None, mol_id=-1):
    if mol_id == -1:
        raise ValueError("ID must be between 1 and 133885")
    for cluster in clusters:
        if prop in cluster:
            model = create_hard_parameter_sharing_model(len(cluster))
            model.load_weights(generate_model_filename(cluster))
            model.compile(optimizer='adam', loss='mse')
            predictions = model.predict([[X_complete[mol_id-1]], [A_complete[mol_id-1]], [E_complete[mol_id-1]]])
            mean, std = 0, 1
            with open(generate_model_helper_filename(prop), 'r') as f:
                lines = f.readlines()
                mean = float(lines[0].strip())
                std = float(lines[1].strip())
            prediction = mean + std * predictions[cluster.index(prop)]
            return prediction[0][0]
    if prop == 'gap':
        lumo = predict_property_neural(prop='lumo', mol_id=mol_id)
        homo = predict_property_neural(prop='homo', mol_id=mol_id)
        return lumo - homo
    raise ValueError("Property was not found in clusters list")

In [7]:
def get_data_folder_path():
    return path.join(getcwd(), '..', '..', 'dsgdb9nsd')

In [8]:
def get_molecule_from_file(filenum):
    filepath = path.join(get_data_folder_path(), 
                           'dsgdb9nsd_' + str(filenum).zfill(6) + '.xyz')
    f = open(filepath, 'r')
    lines = f.readlines()
    f.close()
    num_atoms = int(lines[0])
    atom_list = lines[2:2+num_atoms]
    for i in range(len(atom_list)):
        atom_list[i] = atom_list[i][:atom_list[i].rfind("\t")] + "\n"
    return psi4.geometry("".join(atom_list))

In [9]:
def generate_dft_output_file_path(filenum):
    return path.join('psi4_output', 'output_'+str(filenum)+'.dat')

In [10]:
def process_molecule(filenum, thermochemical=False):
    psi4.core.set_output_file(generate_dft_output_file_path(filenum), False)
    psi4.set_memory('2 GB')
    molecule = get_molecule_from_file(filenum)
    if thermochemical:
        e, wfn = psi4.freq('b3lyp/cc-pvqz', molecule=molecule, return_wfn=True)
    else:
        e, wfn = psi4.energy('b3lyp/cc-pvqz', molecule=molecule, return_wfn=True)
    return wfn

In [11]:
def extract_rotational_constants(filenum, wfn):
    f = open(generate_dft_output_file_path(filenum), 'r')
    lines = f.readlines()
    f.close()
    for i in range(len(lines)-1, -1, -1):
        if lines[i].find('Rotational constants:') > -1 and lines[i].find('[MHz]') > -1:
            words = lines[i].split()
            rot_constants = []
            for const in [words[4], words[7], words[10]]:     
                if const.isnumeric():
                    rot_constants.append(float(const)/1000)
                else:
                    rot_constants.append(const)
                    # rot_constants.append(float(const)/1000)
            return rot_constants
    return None, None, None

In [12]:
def extract_dipole_moment(filenum, wfn):
    f = open(generate_dft_output_file_path(filenum), 'r')
    lines = f.readlines()
    f.close()
    for i in range(len(lines)-1, -1, -1):
        if lines[i].find("Dipole Moment: [D]") > -1:
            return lines[i+1][lines[i+1].find("Total:") + 6:]

In [13]:
def extract_homo_lumo(filenum, wfn):
    homo = wfn.epsilon_a_subset("AO", "ALL").get(wfn.nalpha())
    lumo = wfn.epsilon_a_subset("AO", "ALL").get(wfn.nalpha() + 1)
    return homo, lumo

In [14]:
def extract_gap(filenum, wfn):
    homo, lumo = extract_homo_lumo_gap(fileum, wfn)
    return lumo - homo

In [15]:
def extract_zpve(filenum, wfn):
    f = open(generate_dft_output_file_path(filenum), 'r')
    lines = f.readlines()
    f.close()
    for i in range(len(lines)-1, -1, -1):
        if lines[i].find("Total ZPE, Electronic energy at 0 [K]") > -1:
            words = lines[i].split()
            return words[-2]

In [16]:
def extract_zero_point_internal_energy(filenum, wfn):
    f = open(generate_dft_output_file_path(filenum), 'r')
    lines = f.readlines()
    f.close()
    for i in range(len(lines)-1, -1, -1):
        if lines[i].find("Total E0, Electronic energy") > -1:
            words = lines[i].split()
            return words[-2]

In [17]:
def extract_internal_energy(filenum, wfn):
    f = open(generate_dft_output_file_path(filenum), 'r')
    lines = f.readlines()
    f.close()
    for i in range(len(lines)-1, -1, -1):
        if lines[i].find("Total E, Electronic energy at  298.15 [K]") > -1:
            words = lines[i].split()
            return words[-2]

In [18]:
def extract_enthalpy(filenum, wfn):
    f = open(generate_dft_output_file_path(filenum), 'r')
    lines = f.readlines()
    f.close()
    for i in range(len(lines)-1, -1, -1):
        if lines[i].find("Total H, Enthalpy at  298.15 [K]") > -1:
            words = lines[i].split()
            return words[-2]

In [19]:
def extract_gibbs_free_energy(filenum, wfn):
    f = open(generate_dft_output_file_path(filenum), 'r')
    lines = f.readlines()
    f.close()
    for i in range(len(lines)-1, -1, -1):
        if lines[i].find("Total G,") > -1:
            words = lines[i].split()
            return words[-2]

In [20]:
def extract_cv(filenum, wfn):
    f = open(generate_dft_output_file_path(filenum), 'r')
    lines = f.readlines()
    f.close()
    for i in range(len(lines)-1, -1, -1):
        if lines[i].find('Total Cv') > -1:
            words = lines[i].split()
            return words[2]

In [21]:
def batch_process(start_num, end_num, thermochemical=False):
    f = open("output.csv", "w")
    output_header = "Index,A,B,C,Dipole,HOMO,LUMO"
    if thermochemical:
        output_header += ",zpve,H 298.15,G 298.15"
    output_header += "\n"
    f.write(output_header)
    for filenum in range(start_num, end_num+1):
        wfn = process_molecule(filenum, thermochemical=thermochemical)
        a, b, c = extract_rotational_constants(filenum, wfn)
        dipole = extract_dipole_moment(filenum, wfn)
        homo, lumo = extract_homo_lumo(filenum, wfn)
        output = str(filenum) + "," + str(a) + "," + str(b) + "," + str(c) + "," + str(dipole) + "," + str(homo) + "," + str(lumo)
        if thermochemical:
            zpve = extract_zpve(filenum, wfn)
            enthalpy = extract_enthalpy(filenum, wfn)
            gibbs_free_energy = extract_gibbs_free_energy(filenum, wfn)
            output += "," + str(zpve) + "," + str(enthalpy) + "," + str(gibbs_free_energy)
        output += "\n" 
        f.write(output)
    f.close()

In [22]:
def predict_all_properties_dft(filenum=-1, thermochemical=False):
    if filenum == -1:
        raise ValueError("Filenum must be between 1 and 133885")
    wfn = process_molecule(filenum, thermochemical=thermochemical)
    a, b, c = extract_rotational_constants(filenum, wfn)
    dipole = extract_dipole_moment(filenum, wfn)
    homo, lumo = extract_homo_lumo(filenum, wfn)
    ret_dict = dict()
    ret_dict['A'] = a
    ret_dict['B'] = b
    ret_dict['C'] = c
    ret_dict['mu'] = dipole
    ret_dict['homo'] = homo
    ret_dict['lumo'] = lumo
    ret_dict['gap'] = lumo-homo
    if thermochemical:
        zpve = extract_zpve(filenum, wfn)
        internal_energy = extract_internal_energy(filenum, wfn)
        u0 = extract_zero_point_internal_energy(filenum, wfn)
        enthalpy = extract_enthalpy(filenum, wfn)
        gibbs_free_energy = extract_gibbs_free_energy(filenum, wfn)
        cv = extract_cv(filenum, wfn)
        ret_dict['zpve'] = zpve
        ret_dict['u0'] = u0
        ret_dict['u298'] = internal_energy
        ret_dict['h298'] = enthalpy
        ret_dict['g298'] = gibbs_free_energy
        ret_dict['cv'] = cv
    return ret_dict

In [23]:
def lookup_property(prop=None, mol_id=-1):
    return y_complete.loc[mol_id-1, prop]

In [24]:
def prompt_user_for_calculation():
    while True:
        num = -1
        while num < 1 or num > 133885:
            try:
                num = int(input('Choose a molecule index (1-133885): '))
            except ValueException:
                print("Please provide a valid number")
                num = -1
            
        properties = ['A', 'B', 'C', 'mu', 'homo', 'lumo', 'gap', 'zpve', 'u0', 'u298', 'h298', 'g298', 'cv']
        prop = None
        while prop not in properties:
            print("Choose an available property from the following:")
            print("A, B, C, mu, homo, lumo, gap, zpve, u0, u298, h298, g298, cv")
            prop = input('Choose a property: ')
        
        dft = None
        while dft not in ['0', '1']:
            print("Choose whether to use DFT or neural methods")
            print("0 for neural methods, 1 for DFT")
            dft = input('Calculation type: ')
        
        if dft == '1':
            print('Beginning DFT calculation')
            start = time.time()
            thermochemical = prop in properties[7:]
            ret_dict = predict_all_properties_dft(num, thermochemical=thermochemical)
            print(ret_dict[prop])
            end = time.time()
            print('DFT calculation took', end-start, 's')
        else:
            print('Beginning neural calculation:')
            start = time.time()
            print(predict_property_neural(prop=prop, mol_id=num))
            end = time.time()
            print('Neural method took', end-start, 's')
        print('Actual data:')
        print(lookup_property(prop=prop, mol_id=num))

In [ ]:
prompt_user_for_calculation()

Choose a molecule index (1-133885): 6
Choose an available property from the following:
A, B, C, mu, homo, lumo, gap, zpve, u0, u298, h298, g298, cv
Choose a property: mu
Choose whether to use DFT or neural methods
0 for neural methods, 1 for DFT
Calculation type: 1
Beginning DFT calculation
     2.3408

DFT calculation took 5.908527135848999 s
Actual data:
2.1089
Choose a molecule index (1-133885): 6000
Choose an available property from the following:
A, B, C, mu, homo, lumo, gap, zpve, u0, u298, h298, g298, cv
Choose a property: mu
Choose whether to use DFT or neural methods
0 for neural methods, 1 for DFT
Calculation type: 1
Beginning DFT calculation
     3.1552

DFT calculation took 342.2648570537567 s
Actual data:
2.8755
Choose a molecule index (1-133885): 6000
Choose an available property from the following:
A, B, C, mu, homo, lumo, gap, zpve, u0, u298, h298, g298, cv
Choose a property: u0
Choose whether to use DFT or neural methods
0 for neural methods, 1 for DFT
Calculation type